# Chapter 2: Computational Statistics
___

### Distributions

In [1]:
from thinkbayes import Pmf  # supporting library of the book

In [2]:
pmf = Pmf()  # creates an empty probability mass function

for x in range(1, 7):
    pmf.Set(x, 1/6.0)  # assigns a probability to a value

In [3]:
list(pmf.keys())

[1, 2, 3, 4, 5, 6]

In [4]:
pmf = Pmf()

for w in 'hello how are you my friend truth is you are pretty my friend'.split():
    pmf.Incr(w, 1)  # increases "probability" associated with each word by 1.
                    # actually it is the frequency (a non-normalized probability)

In [7]:
pmf.Prob('you')  # returns the frequecy of the word

2

In [8]:
pmf.Normalize()

13

In [9]:
pmf.Prob('you')  # returns the probability of the word

0.15384615384615385

In the context of Bayesianism, it is natural to use a PMF to map a hypothesis to its probability.

PMF \ $ H_i => P(H_i)$

### The cookie problem

Suppose there are two bowls full of cookies. Bowl 1 has 30 vanilla cookies and 10 chocolate cookies, while Bowl 2 has 20 of each. If I pick a bowl at random, and then a random cookie from it. It turns out to be a **vanilla** cookie.

> What is the probability that it came from Bowl 1?

In [10]:
# info bowl 1: number of cookies per type
vanillas_b1 = 30
chocos_b1 = 10

# info bowl 2: number of cookies per type
vanillas_b2 = 20
chocos_b2 = 20

# 1) knowledge: I take a bowl at random, from which I take a cookie at random
# 2) New data: A vanilla cookie came out.

# 3) Inference: What is the probability that the vanilla cookie came from bowl 1?

In the cookie problem we have 2 hypothesis: 
- $H_1$ : vanilla cookie from bowl 1
- $H_2$ : vanilla cookie from bowl 2

In [33]:
pmf = Pmf()
# encode the prior distributions (before we know what cookie came out)
pmf.Set('Bowl_1', 0.5)  # H1
pmf.Set('Bowl_2', 0.5)  # H2

To update the prior distribution based on **new data** (i.e., we got a vanilla cookie) we multiply each prior by its corresponding **likelihood**:

In [34]:
tag_VANILLA = "vanilla"
tag_CHOCOLATE = "chocolate"

likelihood_H1 = vanillas_b1 / (vanillas_b1 + chocos_b1)
likelihood_H2 = vanillas_b2 / (vanillas_b2 + chocos_b2)

print(f"Likelihood vanilla cookie from bowl 1: {likelihood_H1}")
print(f"Likelihood vanilla cookie from bowl 2: {likelihood_H2}")

pmf.Mult('Bowl_1', likelihood_H1)
pmf.Mult('Bowl_2', likelihood_H2)

Likelihood vanilla cookie from bowl 1: 0.75
Likelihood vanilla cookie from bowl 2: 0.5


After this update, the distribution is no longer normalized, but since we are dealing with **MECE** hypothesis, we can re-normalize:

In [35]:
pmf.Normalize()  # should return 0.625

0.625

The result is a distribution that contains the **posterior probability** for each hypothesis:

In [37]:
pmf.Prob('Bowl_1'), pmf.Prob('Bowl_2')

(0.6000000000000001, 0.4)

Let's rewrite the previous code with classes, to make it more general:

In [38]:
class Cookie(Pmf):
    """ 
    PMF that maps hypotheses to their probabilities.
    Stores the priors and posteriors for each hypothesis given.
    """
    def __init__(self, hypos):
        """ Gives each hypothesis the same prior probability """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

In [39]:
# We have 2 hypotheses:
hypos = ['bowl_1', 'bowl_2']

pmf = Cookie(hypos)

In [40]:
pmf.GetDict()

{'bowl_1': 0.5, 'bowl_2': 0.5}

In [41]:
class Cookie(Pmf):
    """ 
    PMF that maps hypotheses to their probabilities.
    Stores the priors and posteriors for each hypothesis given.
    :param hypos:
    """
    mixes = {
        'bowl_1': {tag_VANILLA: likelihood_H1, tag_CHOCOLATE:1 - likelihood_H1},
        'bowl_2': {tag_VANILLA: likelihood_H2, tag_CHOCOLATE:1 - likelihood_H2}
    }
    def __init__(self, hypos):
        """
        Gives each hypothesis the same prior probability 
        :param hypos: sequence of string bowl IDs
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        """ 
        Returns likelihood of obtaning 'data' given that 'hypo' is true
        :param data: string cookie type
        :param hypo: string bowl ID
        """
        mix = self.mixes.get(hypo)  # from H, get mix of cookies for bowl of that H
        like = mix.get(data)  # get the likelihood of observing the data, if H is true
        return like
    
    def Update(self, data):
        """ 
        Takes some data and updates the probabilities, looping for each H 
        :param data: string cookie type
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

In [42]:
pmf = Cookie(hypos)

new_data = 'vanilla'

pmf.Update(data=new_data)

In [43]:
for hypo, prob in pmf.Items():
    print("Posterior for", hypo,":", prob)

Posterior for bowl_1 : 0.6000000000000001
Posterior for bowl_2 : 0.4


This method has the advantage that it generalizes well to other new data:

In [44]:
datapoints = [tag_VANILLA, tag_CHOCOLATE, tag_VANILLA]  # succesive extraction of cookies

for p in datapoints:
    pmf.Update(p)

In [45]:
for hypo, prob in pmf.Items():
    print("Posterior for", hypo,":", prob)

Posterior for bowl_1 : 0.627906976744186
Posterior for bowl_2 : 0.37209302325581395


<!-- . -->

### The Monty Hall Problem

Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. 

You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to pick door No. 2?" 

Is it to your advantage to switch your choice, or stick to your initial choice?

Source: [Wikipedia](https://en.wikipedia.org/wiki/Monty_Hall_problem)

---
Let's find out using programming.

We create a new class to represent it:

In [71]:
# We have 3 MECE hypotheses:
hypos = ['A', 'B', 'C']  # car is actually behind door A, B, or C, respectively

door_I_chose = hypos[0]  # I choose door A

In [72]:
class Monty(Pmf):
    """ Map: string location of car ==> probability """
    def __init__(self, hypos):
        """
        Initialize the distribution.
        :param hypos: sequence of hypotheses (strings)
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()  # so that each probab is 1 / sum(hypos)
        
    def Likelihood(self, data, hypo):
        """
        Computes the likelihood of the data under the hypothesis.
        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == door_I_chose:
            return 0.5
        else:
            return 1
        
    def Update(self, data):
        """
        Update priors
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

In [73]:
pmf_monty = Monty(hypos)  # I initialize my "distribution of believes" for this problem

In [74]:
# These are the priors (probabilities we assign to each door before we know which door Monty opened
pmf_monty.GetDict()

{'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}

If we see that Monty opens door B, then we can update the priors with the new data, $D$, like:

In [75]:
# After you choose a door, Monty has to open one in which he knows there is no car.
# If the car is behind the door you chose, he opens at random one of the other two rows.
door_monty_opens = 'C'  # Monty opens door B

assert door_monty_opens != door_I_chose, "Cannot be the same doors"
pmf_monty.Update(door_monty_opens)  # I update my beliefes after knowing the door Monty opened

In [76]:
pmf_monty.GetDict()  # check the posteriors

{'A': 0.3333333333333333, 'B': 0.6666666666666666, 'C': 0.0}

Therefore we see that now, If we switch to door B we have a 66% chance of success, and if we don't, we still have a 33% change (i.e., we have not properly utilized the new knowledge of the situation)

### Encapsulating the framwork

Now that we see a pattern in the custom classes we are building to solve our problems (cookie problem and Monty Hall problem), we can create a more genearl class, called `Suite`, that generalizes these features of "Bayesian resoning":

In [112]:
class Suite(Pmf):
    """
    Represents a suite of hypotheses and their probabilities.
    To be inherited by another class for a specicif problem, where the Likelihood
    method needs to be specified.
    """

    def Update(self, data):
        """Updates each hypothesis based on the data.

        data: any representation of the data

        returns: the normalizing constant
        """
        for hypo in list(self.Values()):
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        return self.Normalize()
        
    def Print(self):
        """Prints the hypotheses and their probabilities."""
        print(f"Distribution of probabilites now: \n{'---'*20}\n")
        for hypo, prob in sorted(self.Items()):
            print(f"Hypothesis: {hypo}: {prob}")

And then we use it to define a class for our particular Monty Hall problem:

In [113]:
class Monty(Suite):
    def Likelihood(self, data, hypo):
        """
        Computes the likelihood of the data under the hypothesis.
        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == door_I_chose:
            return 0.5
        else:
            return 1

In [114]:
suite = Monty(['A', 'B', 'C'])
suite.GetDict()

{'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}

In [115]:
suite.Update('B')

suite.Print()

Distribution of probabilites now: 
------------------------------------------------------------

Hypothesis: A: 0.3333333333333333
Hypothesis: B: 0.0
Hypothesis: C: 0.6666666666666666


### The M&M Problem

Before 1995, the color mix in a bag of plain M&M’s was 
- 30% Brown
- 20% Yellow
- 20% Red
- 10% Green
- 10% Orange
- 10% Tan

Afterward it was 
- 24% Blue
- 20% Green
- 16% Orange
- 14% Yellow
- 13% Red
- 13% Brown

Suppose a friend of mine has two bags of M&M’s, and he tells me that one is from 1994 and one from 1996. **He won’t tell me which is which**, but he gives me one M&M from each bag. **The one from the first bag is yellow and the one from the second bag is green**. 

> What is the probability that the yellow one came from the 1994 bag?

This problem is similar to the cookie problem, with the twist that I draw one sample from each bowl/bag.

In [123]:
# observations
obs1 = ('bag1', 'yellow')  # got a yellow M&M from first bag (maybe this bag is from 94?)
obs2 = ('bag2', 'green')   # got a green M&M from second bag (maybe this bag is from 94?)

observations = [obs1, obs2]

In [120]:
class M_and_M(Suite):
    """Map from hypothesis (A or B) to probability."""
    # known information regarding the distribution of colors in each type of bag
    mix94 = dict(brown=30, yellow=20, red=20, green=10, orange=10, tan=10, blue=0)
    mix96 = dict(blue=24, green=20, orange=16, yellow=14, red=13, brown=13, tan=0)
    # MECE hypotheses
    hypoA = dict(bag1=mix94, bag2=mix96)  # the 1st bag has the mix of 94
    hypoB = dict(bag1=mix96, bag2=mix94)  # the 1st bag has the mix of 96

    hypotheses = dict(A=hypoA, B=hypoB)

    def Likelihood(self, data, hypo):
        """
        Computes the likelihood of the data under the hypothesis.
        hypo: string hypothesis (A or B)
        data: tuple of string bag, string color
        """
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

First, we only have our prior:

In [122]:
# H_A: the 1st bag has the mix of 94
# H_B: the 1st bag has the mix of 96
suite = M_and_M(['A', 'B'])
suite.Print()

Distribution of probabilites now: 
------------------------------------------------------------

Hypothesis: A: 0.5
Hypothesis: B: 0.5


In [124]:
# Updating of priors
for obs in observations:
    suite.Update(obs)
    
suite.Print()

Distribution of probabilites now: 
------------------------------------------------------------

Hypothesis: A: 0.7407407407407407
Hypothesis: B: 0.2592592592592592


Which means that now, we are 74% sure that our hypothesis A is true, namely, **we are 74% sure that the 1st bag has the color mix of 94.** 

Our question was 'What is the probability that the yellow one came from the 1994 bag?'. Since we took the yellow M&M from the first bag (observation 1), this is equivalent to saying that **there is a 74% chance that the yellow M&M came from the 1994 bag.**

And this, indeed, agrees with our theoretical (by hand) results using the table method.